### import...

In [3]:
import torch
import torchvision
import torch.nn as nn ## Neural Network API
import torch.nn.functional as F 
'''
torch.nn은 nn.Module classes를 정의하지만 nn.functional은 말그대로 functional(stateless) 하게 접근한다
다시 말해서 nn.Modules는 python class로 정의가 되고 따라서 attribute들이 존재한다
ex. nn.Conv2d는 self.weight와 같은 internal attribute들이 있다

반면에 F.conv2d는 그냥 operation이 정의되는 것이고 그냥 연산을 수행한다. internal attribute 없음
그래서 stateless objesct에 forward 연산을 할 때는 functional API를 쓴다.
ex. F.relu : nn.ReLU는 아무런 파라미터도 저장하지 않기때문에 굳이 모듈로 정의할 필요가 없음
'''
import torch.optim as optim ## Optimizer algorithm (SGD, Adam, Adagrad...)
from torch.utils.data import DataLoader ## Dataset 관리, batch 생성을 도와주는 함수
import torchvision.datasets as datasets ## MNIST, CIFAR10 등 benchmark dataset 관련
import torchvision.transforms as transforms ## 이미지 데이터에 전처리하는 함수 모음 
from torch.utils.tensorboard import SummaryWriter ## tensorboard에 출력하는 함수
import torch.backends.cudnn as cudnn # cudnn 관련

from torchsummary import summary ## model의 구조, 파라미터 수 등을 알 수 있게 해줌
import torch.onnx ## torch model을 onnx로 변환하기 위함

### GPU Setting

In [4]:
# cuda 사용 가능한지 

print(torch.cuda.is_available())

True


In [5]:
# 일반적으로 device 지정해줄 때 사용

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# Multi GPU 사용할 때
# 연구실 서버 기준 GPU 4개 중에서 0, 2, 3 가능하다면

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 2, 3'

In [7]:
# 현재 사용가능한 GPU 개수 확인

print(torch.cuda.device_count())

1


In [8]:

# 일반적으로 multi GPU 사용할 때, GPU 할당

os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(list(map(str, list(range(torch.cuda.device_count())))))

In [9]:
# cudnn 설정, GPU를 사용하고 있으면 cudnn.enabled = True
import torch.backends.cudnn as cudnn

print(cudnn.enabled)

True


In [10]:
# inbuild cudnn auto-tuner가 사용 중인 하드웨어에 가장 적합한 알고리즘을 선택할 수 있게 허용

cudnn.benchmark = True

In [11]:
# GPU device의 사용가능한 메모리 확인하기

# byte 기준
print(torch.cuda.get_device_properties('cuda:0').total_memory, ' byte')

# mega byte 기준
print(torch.cuda.get_device_properties('cuda:0').total_memory // 1e6, ' MB')

# giga byte 기준
print(torch.cuda.get_device_properties('cuda:0').total_memory // 1e9, ' GB')

11553341440  byte
11553.0  MB
11.0  GB


In [13]:
# Multi GPU 사용할 때 메모리 확인

gpu_ids = list(map(str, list(range(torch.cuda.device_count()))))
total_gpu_memory = 0
for gpu_id in gpu_ids:
    total_gpu_memory += torch.cuda.get_device_properties('cuda:'+gpu_id).total_memory

### data.cuda(non_blocking=True) 사용
- 뒤에서 `torch.utils.data.DataLoader`의 `pin_memory`를 사용할 때 일반적으로 사용하는 옵션

In [ ]:
'''
for i, (images, target) in enumerate(train_loader):
    data_time.update(time.time() - end)

    if args.gpu is not None:
        images = images.cuda(args.gpu, non_blocking=True)

    if torch.cuda.is_available():
        target = target.cuda(args.gpu, non_blocking=True)

    output = model(images)
    loss = criterion(output, target)

'''

위에서 볼 수 있듯이 train_loader의 data를 GPU로 보내줄 때 `.cuda()`로 변환함

이 때, `non_blocking=True` 를 옵션으로 지정해주게 되면 비동기식으로 GPU에 데이터를 전달할 수 있게 된다.